In [1]:
import numpy as np 
import pandas as pd

### Read data

In [2]:
df = pd.read_csv('../../data/raw/all_teams_data.csv')
pd.options.display.max_columns = None
df.head()

In [3]:
df['Comp'].value_counts()

In [4]:
import numpy as np

# Replace empty strings with NaN
df.replace('', np.nan, inplace=True)

# Calculate the percentage of missing values
missing_percentage = df.isnull().mean() * 100
missing_percentage[missing_percentage > 10]

### La Liga data

In [5]:
df = df[df['Comp'] == 'La Liga']

In [6]:
df.columns.to_list()

In [7]:
# columns_to_drop = [col for col in df.columns if col.startswith('Expected') or col.startswith('x')]

# df.drop(columns=columns_to_drop, inplace=True)
# df.head()

In [8]:
%cd ../

In [9]:
from preprocessing.preprocess import *
df2 = preprocess_data(df)
df2.sort_values(by='Time', inplace=True)

In [10]:
object_columns = df2.select_dtypes(include=['object']).columns
print(object_columns)

In [11]:
# df2.drop(columns = ['Captain', 'Formation', 'Opp Formation'], inplace=True)

In [12]:
import numpy as np

# Replace empty strings with NaN
df2.replace('', np.nan, inplace=True)

# Calculate the percentage of missing values
missing_percentage = df2.isnull().mean() * 100
sorted(missing_percentage[missing_percentage > 0].items(), key=lambda x: x[1], reverse=True)

G/SoT missing because SoT = 0

Save% missing because SoT = 0

#### How many seasons are there ?

In [13]:
num_seasons = df2['Season'].nunique()
print(f'There are {num_seasons} seasons in the data.')

#### How many teams are there ?

In [14]:
num_teams = df2['Team'].nunique()
print(f'There are {num_teams} teams in the data.')

In [15]:
from unidecode import unidecode

df2['Opponent'] = df2['Opponent'].apply(unidecode)

In [16]:
df2.loc[df2['Opponent'] == 'La Coruna', 'Opponent'] = 'Deportivo La Coruna'
df2.loc[df2['Opponent'] == 'Betis', 'Opponent'] = 'Real Betis'

#### How many decisive games ?

In [17]:
decisive_games = df2[df2['Result'] != 'D'].shape[0]
print(f'There are {decisive_games / 2} decisive games in the data.')

#### Overall results of each team through out 8 seasons

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count the number of wins, draws, and losses for each team
results = df2.groupby('Team')['Result'].value_counts().unstack().fillna(0)
results.columns = ['Draws', 'Losses', 'Wins']
# Calculate total matches and win, draw, loss percentages
results['Total Matches'] = results.sum(axis=1)
results['Win %'] = (results['Wins'] / results['Total Matches']) * 100
results['Draw %'] = (results['Draws'] / results['Total Matches']) * 100
results['Loss %'] = (results['Losses'] / results['Total Matches']) * 100

# Sort the columns order
results = results[['Total Matches', 'Wins', 'Win %', 'Draws', 'Draw %', 'Losses', 'Loss %']]

# Add 'Team' column
results = results.reset_index()

Which teams have the highest win %

In [19]:
results.sort_values(by='Win %', ascending=False)

Which teams score the most goals ?

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count the number of goals for, goals against, goals diff of each team
team_goals = df2.groupby('Team').agg({
    'GF': 'sum',
    'GA': 'sum'
}).reset_index()

# Calculate Goal Difference (GD)
team_goals['GD'] = team_goals['GF'] - team_goals['GA']

# Sort the teams by Goal Difference
team_goals = team_goals.sort_values(by='GD', ascending=False)

team_goals

Final table of each seasons

In [21]:
import pandas as pd
import matplotlib.pyplot as plt

for season in sorted(df2['Season'].unique()):
    table = df2[df2['Season'] == season].groupby('Team')
    result_sum = table['Points'].sum().sort_values(ascending=False)
    print(f'Season {season} - Total Results: {result_sum}')
    num_matches = table['Points'].count()
    if num_matches.iloc[0] != 38:
        print(f"Warning: The number of matches for season {season} is {num_matches.iloc[0]}, which is different from 38.")

In [22]:
import matplotlib.pyplot as plt

# Plot the first 5 best teams
plt.figure(figsize=(14, 8))
for team in results.sort_values(by='Win %', ascending=False)['Team'].head(5):
    team_data = df2[df2['Team'] == team]
    win_percentage = team_data.groupby('Season')['Result'].apply(lambda x: (x == 'W').mean() * 100)
    plt.plot(win_percentage.index, win_percentage.values, label=team)

plt.xlabel('Seasons')
plt.ylabel('Win %')
plt.title('Win Percentage of Top 5 Teams Over Seasons')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True)
plt.show()

#### Describe univariate variables

In [23]:
prematch_cols = ['Time',
 'Comp',
 'Round',
 'Day',
 'Venue',
 'Opponent',
 'Captain',
 'Formation',
 'Opp Formation',
 'Referee',
 'Season',
 'Team']
shooting_cols = ['Standard__Gls',
 'Standard__Sh',
 'Standard__SoT',
 'Standard__SoT%',
 'Standard__G/Sh',
 'Standard__G/SoT',
 'Standard__Dist',
 'Standard__FK',
 'Standard__PK',
 'Standard__PKatt']
goal_keeping_cols = ['Performance__SoTA',
 'Performance__GA',
 'Performance__Saves',
 'Performance__Save%',
 'Performance__CS',
 'Performance__PSxG',
 'Performance__PSxG+/-',
 'Penalty Kicks__PKatt',
 'Penalty Kicks__PKA',
 'Penalty Kicks__PKsv',
 'Penalty Kicks__PKm',
 'Launched__Cmp',
 'Launched__Att',
 'Launched__Cmp%',
 'Passes__Att (GK)',
 'Passes__Thr',
 'Passes__Launch%',
 'Passes__AvgLen',
 'Goal Kicks__Att',
 'Goal Kicks__Launch%',
 'Goal Kicks__AvgLen',
 'Crosses__Opp',
 'Crosses__Stp',
 'Crosses__Stp%',
 'Sweeper__#OPA']
passing_cols = ['Total__Cmp',
 'Total__Att',
 'Total__Cmp%',
 'Total__TotDist',
 'Total__PrgDist',
 'Short__Cmp',
 'Short__Att',
 'Short__Cmp%',
 'Medium__Cmp',
 'Medium__Att',
 'Medium__Cmp%',
 'Long__Cmp',
 'Long__Att',
 'Long__Cmp%',
 'Ast',
 'KP',
 '1/3',
 'PPA',
 'CrsPA',
 'PrgP']
pass_type_cols = ['Att',
 'Pass Types__Live',
 'Pass Types__Dead',
 'Pass Types__FK',
 'Pass Types__TB',
 'Pass Types__Sw',
 'Pass Types__Crs',
 'Pass Types__TI',
 'Pass Types__CK',
 'Corner Kicks__In',
 'Corner Kicks__Out',
 'Corner Kicks__Str',
 'Outcomes__Cmp',
 'Outcomes__Off',
 'Outcomes__Blocks']
goal_shot_creation_cols = ['SCA Types__SCA',
 'SCA Types__PassLive',
 'SCA Types__PassDead',
 'SCA Types__TO',
 'SCA Types__Sh',
 'SCA Types__Fld',
 'SCA Types__Def',
 'GCA Types__GCA',
 'GCA Types__PassLive',
 'GCA Types__PassDead',
 'GCA Types__TO',
 'GCA Types__Sh',
 'GCA Types__Fld',
 'GCA Types__Def']
defensive_actions_cols = ['Tackles__Tkl',
 'Tackles__TklW',
 'Tackles__Def 3rd',
 'Tackles__Mid 3rd',
 'Tackles__Att 3rd',
 'Challenges__Tkl',
 'Challenges__Att',
 'Challenges__Tkl%',
 'Challenges__Lost',
 'Blocks__Blocks',
 'Blocks__Sh',
 'Blocks__Pass',
 'Int',
 'Tkl+Int',
 'Clr',
 'Err']
possession_cols = ['Poss',
 'Touches__Touches',
 'Touches__Def Pen',
 'Touches__Def 3rd',
 'Touches__Mid 3rd',
 'Touches__Att 3rd',
 'Touches__Att Pen',
 'Touches__Live',
 'Take-Ons__Att',
 'Take-Ons__Succ',
 'Take-Ons__Succ%',
 'Take-Ons__Tkld',
 'Take-Ons__Tkld%',
 'Carries__Carries',
 'Carries__TotDist',
 'Carries__PrgDist',
 'Carries__PrgC',
 'Carries__1/3',
 'Carries__CPA',
 'Carries__Mis',
 'Carries__Dis',
 'Receiving__Rec',
 'Receiving__PrgR']
miscellaneous_cols = ['Performance__CrdY',
 'Performance__CrdR',
 'Performance__2CrdY',
 'Performance__Fls',
 'Performance__Fld',
 'Performance__Off',
 'Performance__Crs',
 'Performance__Int',
 'Performance__TklW',
 'Performance__PKwon',
 'Performance__PKcon',
 'Performance__OG',
 'Performance__Recov',
 'Aerial Duels__Won',
 'Aerial Duels__Lost',
 'Aerial Duels__Won%']

In [24]:
df2[prematch_cols].head()

In [25]:
df2.describe()

#### Check categorical variable

In [26]:
import seaborn as sns

sns.countplot(data=df2, x='Venue', hue='Result')
plt.title('Result Distribution by Venue')
plt.show()

In [27]:
# Calculate win percentage for each team under different referees
referee_performance = df2.groupby(['Referee', 'Team'])['Result'].apply(lambda x: (x == 'W').mean() * 100).reset_index()

# Find the referee with the highest win percentage for each team
best_referee_for_team = referee_performance.loc[referee_performance.groupby('Team')['Result'].idxmax()]

# Rename columns for clarity
best_referee_for_team.columns = ['Referee', 'Team', 'Win %']

best_referee_for_team.sort_values(by='Win %', ascending=False)

In [28]:
# Calculate win percentage for each formation
formation_win_percentage = df2.groupby('Formation')['Result'].apply(lambda x: (x == 'W').mean() * 100).reset_index()

# Rename columns for clarity
formation_win_percentage.columns = ['Formation', 'Win %']

# Sort the formations by win percentage
formation_win_percentage = formation_win_percentage.sort_values(by='Win %', ascending=False)

formation_win_percentage

#### Check numerical variables

In [29]:
for cols in shooting_cols:
    sns.histplot(data=df2, x=cols, hue='Result', multiple='stack')
    plt.title(cols)
    plt.show()

#### Calculate Correlations

In [30]:
# Map the 'Result' column to numerical values
# result_mapping = {'W': 2, 'D': 1, 'L': 0}
# df2['Result_num'] = df2['Result'].map(result_mapping)
df2['GD'] = df2['GF'] - df2['GA']
# Select only numeric columns
numeric_df2 = df2.drop(columns=prematch_cols).select_dtypes(include=[np.number])

# Calculate the correlation matrix
correlation_matrix = numeric_df2.corr()

result_correlation = correlation_matrix['GD'].round(3).sort_values(ascending=False)
pd.options.display.max_rows = None
result_correlation

Correlation of shooting stats with goals

In [31]:
correlation_matrix['GF'][shooting_cols]

Correlation of goal keeping stats with goals against

In [32]:
correlation_matrix['GA'][goal_keeping_cols]

In [33]:
df2.reset_index(drop=True, inplace=True)
df2.info()

In [34]:
# df2.to_csv('D:\dev\project\Football-Match-Prediction\data\processed/la_liga_data.csv', index=False)

In [35]:
df2 = pd.read_csv('D:\dev\project\Football-Match-Prediction\data\processed/la_liga_data.csv')
df2.head()

Drop columns with correlation with GD lower than 0.1

In [36]:
threshold = 0.2  # Set your threshold value here

# Get the correlation values for the 'GD' column
gd_correlation = correlation_matrix['GD']

# Find columns with correlation lower than the threshold
low_correlation_cols = gd_correlation[gd_correlation.abs() < threshold].index

# Drop these columns from the dataframe
df2.drop(columns=low_correlation_cols, inplace=True)

print(f"Dropped columns with correlation lower than {threshold}: {low_correlation_cols.tolist()}")

In [37]:
import numpy as np

# Select only numeric columns
numeric_df2 = df2.select_dtypes(include=[np.number])

# Calculate the correlation matrix
corr_matrix = numeric_df2.corr().abs()

# Select the upper triangle of the correlation matrix
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with high correlation (exceeding 0.9)
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

# Drop the redundant features
df2.drop(columns=to_drop, inplace=True)

print(f"Removed redundant features: {to_drop}")

In [38]:
df2.columns.to_list()

In [39]:
import numpy as np

# Calculate the percentage of missing values
missing_percentage = df2.isnull().mean() * 100
sorted(missing_percentage[missing_percentage > 0].items(), key=lambda x: x[1], reverse=True)

In [40]:
object_columns = df2.select_dtypes(include=['object']).columns
print(object_columns)

In [41]:
df2.drop(columns=['Referee', 'Captain', 'Comp', 'Formation', 'Opp Formation'], inplace=True)

In [42]:
for col in df2.select_dtypes(include=[np.number]).columns:
    df2[col] = df2[col].fillna(df2.groupby('Team')[col].transform('mean'))
    # df2[cols] = df2[cols].fillna(df2.groupby('Team')[cols].mean()) 

In [43]:
import numpy as np

# Calculate the percentage of missing values
missing_percentage = df2.isnull().mean() * 100
sorted(missing_percentage[missing_percentage > 0].items(), key=lambda x: x[1], reverse=True)

## Feature Engineering

In [44]:
feature_cols = [col for col in df2.columns if col not in prematch_cols]
feature_cols.remove('Result')
categorical_cols = [col for col in feature_cols if col not in df2.select_dtypes(include=[np.number]).columns]
print("Categorical columns:", categorical_cols)

In [45]:
to_drop = ['Referee', 'Captain', 'Day', 'Comp', 'Season', 'Formation', 'Opp Formation']
for cols in to_drop: 
    prematch_cols.remove(cols)

In [46]:
# Sort the dataframe by time
df2.sort_values(by='Time', inplace=True)

# Modify the function to return only the weighted average columns
def calculate_weighted_avg_excluding_current(group, feature_cols, weight_col='Round'):
    weighted_avgs = pd.DataFrame(index=group.index)
    for col in feature_cols:
        cumulative_weighted = (group[col] * group[weight_col]).cumsum().shift(1)
        cumulative_weights = group[weight_col].cumsum().shift(1)
        weighted_avg = cumulative_weighted / cumulative_weights
        weighted_avgs[col + '_WeightedAvg'] = weighted_avg #.fillna(0)
    return weighted_avgs

# Apply the function to each team group
weightedAvg = df2.groupby('Team', group_keys=False).apply(calculate_weighted_avg_excluding_current, feature_cols=feature_cols)

# Display the DataFrame with weighted averages
weightedAvg.tail()

#### Estimate current form by avg of last n rounds

In [47]:
# Define the window size for rolling averages
window_size = 5

# Function to calculate rolling averages
def calculate_rolling_averages(group):
    return group[feature_cols].shift().rolling(window=window_size, min_periods=1).mean()

# Apply the function to df2 grouped by 'Team'
rolling_averages = df2.groupby('Team', group_keys=False).apply(calculate_rolling_averages)

# Rename columns to indicate rolling averages
rolling_averages.columns = [f"{col}_avg" for col in rolling_averages.columns]
rolling_averages.tail()

In [48]:
# Combine rolling averages with the original dataframe
training_df = pd.concat([df2[prematch_cols].reset_index(drop=True), weightedAvg.reset_index(drop=True), df2[['GF', 'GA', 'Result']].reset_index(drop=True)], axis=1)

# Drop rows with missing values
# training_df.dropna(inplace=True)

# # Your training data is now ready
training_df.head()

In [49]:
training_df.drop(columns=['Round'], inplace=True)

In [50]:
training_df.dropna(inplace=True)

In [51]:
training_df.info()

In [52]:
# Rename columns to include 'HomeTeam_' prefix where Venue == 1
df_team = training_df[training_df['Venue'] == 1].copy()
df_team['HomeTeam'] = df_team['Team']
df_team.drop(columns=['Team'], inplace=True)
df_team['AwayTeam'] = df_team['Opponent']
df_team.drop(columns=['Opponent'], inplace=True)
df_team = df_team.rename(columns=lambda x: f"HomeTeam_{x}" if x not in ['Time', 'Venue', 'HomeTeam', 'AwayTeam'] else x)
df_team.drop(columns=['Venue'], inplace=True)
df_opponent = training_df[training_df['Venue'] == 0].copy()
df_opponent['AwayTeam'] = df_opponent['Team']
df_opponent = df_opponent.drop(columns=['Team'])
df_opponent['HomeTeam'] = df_opponent['Opponent']
df_opponent = df_opponent.drop(columns=['Opponent'])
df_opponent = df_opponent.rename(columns=lambda x: f"AwayTeam_{x}" if x not in ['Time', 'Venue', 'HomeTeam', 'AwayTeam'] else x)
df_opponent.drop(columns=['Venue'], inplace=True)   
# Merge the two dataframes
df_merged = pd.merge(df_team, df_opponent, on=['Time', 'HomeTeam', 'AwayTeam'])

# Calculate head-to-head statistics

# Ensure training_df is sorted by 'Time'
training_df.sort_values('Time', inplace=True)

# Initialize a list to store head-to-head stats
h2h_stats = []

# Iterate over each match in df_merged
for index, row in df_merged.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    match_time = row['Time']
    
    # Filter past matches between the two teams
    past_matches = training_df[
        (((training_df['Team'] == home_team) & (training_df['Opponent'] == away_team)) |
         ((training_df['Team'] == away_team) & (training_df['Opponent'] == home_team))) &
        (training_df['Time'] < match_time)
    ]
    
    total_matches = len(past_matches)
    if total_matches == 0:
        home_wins = away_wins = draws = home_goals = away_goals = 0
    else:
        # Results from the perspective of the home team
        home_wins = ((past_matches['Team'] == home_team) & (past_matches['Result'] == 'W')).sum()
        away_wins = ((past_matches['Team'] == home_team) & (past_matches['Result'] == 'L')).sum()
        draws = (past_matches['Result'] == 'D').sum()
        
        # Goals scored by each team
        home_goals = past_matches.loc[past_matches['Team'] == home_team, 'GF'].sum() + \
                     past_matches.loc[past_matches['Opponent'] == home_team, 'GA'].sum()
        away_goals = past_matches.loc[past_matches['Team'] == away_team, 'GF'].sum() + \
                     past_matches.loc[past_matches['Opponent'] == away_team, 'GA'].sum()
    
    h2h_stats.append({
        'H2H_Total_Matches': total_matches,
        'H2H_Home_Wins': home_wins,
        'H2H_Away_Wins': away_wins,
        'H2H_Draws': draws,
        'H2H_Home_Goals': home_goals,
        'H2H_Away_Goals': away_goals,
    })

# Convert the list to a DataFrame
h2h_stats_df = pd.DataFrame(h2h_stats)

# Merge the head-to-head stats with the original dataframe
df_merged = pd.concat([df_merged.reset_index(drop=True), h2h_stats_df], axis=1)

In [53]:
df_merged.head()

In [54]:
df_merged.info()

In [55]:
df_merged.drop(columns=['HomeTeam_GA', 'AwayTeam_GA', 'AwayTeam_Result'], inplace=True)

In [56]:
# df_merged.to_csv('D:\dev\project\Football-Match-Prediction\data\processed\df_merged.csv', index=False)

In [57]:
sns.countplot(data=df_merged, x='HomeTeam_Result')